# Vector Search

Quick Example of Vector Search on Product Nodes


## Setup

In [1]:
import graphdatascience
graphdatascience.__version__

'1.8'

In [2]:
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings, BedrockEmbeddings, SentenceTransformerEmbeddings

In [3]:
load_dotenv('.env', override=True)

True

In [4]:
def load_embedding_model(embedding_model_name: str):
    if embedding_model_name == "openai":
        embeddings = OpenAIEmbeddings()
        dimension = 1536
    elif embedding_model_name == "aws":
        embeddings = BedrockEmbeddings()
        dimension = 1536
    else:
        embeddings = SentenceTransformerEmbeddings(
            model_name="all-MiniLM-L6-v2", cache_folder="/embedding_model"
        )
        dimension = 384
    return embeddings, dimension

In [5]:
embedding_model, dimension = load_embedding_model(os.getenv('EMBEDDING_MODEL'))

In [6]:
# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

In [7]:
gds.version()

'2.5.1+39'

## Product Search
Basic product search using vector index

### Simple Vector Search

In [8]:
#search_prompt = 'denim jeans, loose fit, high-waist'
search_prompt = 'winter sweater with zipper'

In [9]:
query_vector = embedding_model.embed_query(search_prompt)
print(f'query vector length: {len(query_vector)}')
print(f'query vector sample: {query_vector[:10]}')

query vector length: 1536
query vector sample: [-0.015749799025370704, -0.015950007301089343, -0.0007332665816759234, -0.020714988850108796, -0.02581365210042274, 0.013814441805101402, -0.009436531886400323, -0.010190653574407153, 0.0030214921451847962, -0.019353566614757605]


In [10]:
gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.prodName AS prodName,
    product.productTypeName AS productTypeName,
    product.garmentGroupName AS garmentGroupName,
    product.detailDesc AS detailDesc,
    score
''', params={'queryVector': query_vector})

,prodName,productTypeName,garmentGroupName,detailDesc,score
0,Catfish Zip,Sweater,Knitwear,"Soft, textured-knit jumper with a ribbed stand...",0.933451
1,Yolo Zip L/S,Sweater,Knitwear,"Jumper in a soft, rib knit with a stand-up col...",0.931802
2,Ben zip hoodie,Sweater,Knitwear,Fine-knit jacket in a soft viscose blend with ...,0.931022
3,BANANA HALF ZIP SWEATER,Sweater,Knitwear,Jumper in a soft rib knit with a high stand-up...,0.930954
4,Raven Half Zip Sweater,Sweater,Jersey Fancy,"Sports top in stretch, fast-drying functional ...",0.930447
5,Southern Sweater,Sweater,Knitwear,Jumper in a soft rib knit with pointelle detai...,0.929159
6,Zorro half-zip sweater,Sweater,Jersey Basic,"Short top in sweatshirt fabric with a high, ri...",0.928725
7,BAY BLOCK STRIPE ZIP-UP,Sweater,Knitwear,"Jumper in a soft, fine-knit modal and cotton b...",0.928706
8,Yolo Zip LS,Sweater,Knitwear,"Fitted jumper in a soft, rib knit with a turtl...",0.927682
9,Håkan half zip knit,Sweater,Knitwear,"Jumper in a soft cotton knit with a high, ribb...",0.927454
